<a href="https://colab.research.google.com/github/AbhinavShakunt/MY_Data_Science_Notebooks/blob/main/Distributedm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="4"


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [4]:
tf.__version__

'2.15.0'

In [5]:
tf.test.is_built_with_cuda()

True

In [6]:
(X_train, y_train), (X_test,y_test) = tf.keras.datasets.cifar10.load_data()

170498071/170498071 [==============================] - 13s 0us/step


In [7]:
X_train.shape

(50000, 32, 32, 3)

In [8]:
y_train.shape

(50000, 1)

In [9]:
classes = ["airplane","automobile","bird","cat","deer","dog","frog","horse","ship","truck"]

In [10]:
classes[y_train[3][0]]

'deer'

In [11]:
y_train[:3]

array([[6],
       [9],
       [9]], dtype=uint8)

In [12]:
y_test.shape

(10000, 1)

In [13]:
X_test.shape

(10000, 32, 32, 3)

In [14]:
X_train_scaled = X_train / 255
X_test_scaled = X_test / 255

In [15]:
y_train_categorical = keras.utils.to_categorical(
    y_train, num_classes=10, dtype='float32'
)
y_test_categorical = keras.utils.to_categorical(
    y_test, num_classes=10, dtype='float32'
)

In [16]:
y_train[0:5]

array([[6],
       [9],
       [9],
       [4],
       [1]], dtype=uint8)

In [17]:
y_train_categorical[0:5]

array([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [18]:
def get_model():
    model = keras.Sequential([
            keras.layers.Flatten(input_shape=(32,32,3)),
            keras.layers.Dense(3000, activation='relu'),
            keras.layers.Dense(1000, activation='relu'),
            keras.layers.Dense(10, activation='sigmoid')
        ])

    model.compile(optimizer='SGD',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [19]:
train_tf_dataset = tf.data.Dataset.from_tensor_slices((X_train_scaled, y_train_categorical))
test_tf_dataset = tf.data.Dataset.from_tensor_slices((X_test_scaled, y_test_categorical))

In [20]:
strategy = tf.distribute.MirroredStrategy()

In [21]:
strategy.num_replicas_in_sync

1

In [22]:
BATCH_SIZE_PER_REPLICA = 250
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync


train_dataset = train_tf_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_tf_dataset.batch(BATCH_SIZE)

In [23]:
%%timeit -n1 -r1
with strategy.scope():
    gpu_model = get_model()
    gpu_model.fit(train_dataset, epochs=2)

Epoch 1/2
200/200 [==============================] - 41s 202ms/step - loss: 1.9724 - accuracy: 0.3010
Epoch 2/2
200/200 [==============================] - 41s 205ms/step - loss: 1.8114 - accuracy: 0.3656
1min 25s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [24]:
%%timeit -n1 -r1
with tf.device('/CPU:0'):
    cpu_model = get_model()
    cpu_model.fit(train_dataset, epochs=2)

Epoch 1/2
200/200 [==============================] - 42s 208ms/step - loss: 1.9856 - accuracy: 0.2949
Epoch 2/2
200/200 [==============================] - 41s 204ms/step - loss: 1.8198 - accuracy: 0.3632
2min 4s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
